Getting to know your computerGetting to know your computer
--

1
--

In [1]:
parse(Float64, "1101011101010000")

1.10101110101e15

2
--

In [2]:
A = [5 3; 0 5]

2×2 Array{Int64,2}:
 5  3
 0  5

In [3]:
eig(A)

([5.0,5.0],
[1.0 -1.0; 0.0 3.70074e-16])

In [4]:
cond(A)

1.8064183905346336

In [5]:
using Distributions
using Benchmarks

B = reshape(rand(Normal(),1000000),1000,1000);

function manualSum(matrix::Array{Float64,2}, rowMajor::Bool)
    intersum = 0.
    
    if rowMajor
        for i in 1:size(matrix, 1)
            for j in 1:size(matrix, 2)
                intersum += matrix[i, j]
            end
        end
    else
        for i in 1:size(matrix, 1)
            for j in 1:size(matrix, 2)
                intersum += matrix[j, i]
            end
        end
    end
    
    return intersum
end

manualSum (generic function with 1 method)

In [6]:
@benchmark manualSum(B, true)

================ Benchmark Results ========================
     Time per evaluation: 6.64 ms [6.49 ms, 6.80 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 1.17 s


In [7]:
@benchmark manualSum(B, false)

================ Benchmark Results ========================
     Time per evaluation: 1.85 ms [1.75 ms, 1.94 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.68 s


Julia is column-major!

In [8]:
@benchmark sum(B)

================ Benchmark Results ========================
     Time per evaluation: 1.62 ms [1.48 ms, 1.76 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.70 s


Writing a good for-loop is as good as using built-in functions!

Root finding and optimization
--
1
--

In [9]:
function myBisection(f, a, b; tol=10e-5)
    @assert f(a)*f(b) < 0
    x = 0.
    while (abs(a-b) > tol)
        x = (a + b)/2
        if (f(x)*f(a) < 0)
            b = x
        else
            a = x
        end
    end
    
    return x
end

myBisection (generic function with 1 method)

In [10]:
myBisection(sin,-0.5,1)

-3.0517578125e-5

2
--


In [11]:
using Optim

In [12]:
optimize(sin, -2.0, 1.0)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-2.000000, 1.000000]
 * Minimizer: -1.570796e+00
 * Minimum: -1.000000e+00
 * Iterations: 9
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 10

3
--
Normalize the price of apples to 1 and find the price of bananas, p, which clears the market.

In [13]:
const w=0.9
const x=-0.5

u_A(cons) = (w*cons[1]^x + (1-w)*cons[2]^x)^(1/x)
u_B(cons) = (w*cons[2]^x + (1-w)*cons[1]^x)^(1/x)

u_B (generic function with 1 method)

In [52]:
function HHproblem(u, endowment, p)
    
    wealth = (endowment' * [0, p])[1] #total wealth in terms of money
    objective(percent) = -u([percent*wealth/1, (1-percent)*wealth/p]) #percentage of wealth spent on apples
        
    optimum = optimize(objective, 0.0, 1.0).minimizer
    
    return [optimum*wealth/1, (1-optimum)*wealth/p]
end

HHproblem (generic function with 1 method)

In [62]:
function marketClearing(p; end_A = [0, 5], end_B = [10, 0])
    HHproblem(u_A, end_A, p)[1] + HHproblem(u_B, end_B, p)[1] - end_A[1] - end_B[1]
end

marketClearing (generic function with 1 method)

In [68]:
p_star = myBisection(marketClearing, 0.1, 100)

2.6387146949768066

In [77]:
marketClearing2(p) = marketClearing(p, end_B = [0, 5])
p_star2 = myBisection(marketClearing2, 0.1, 100)

LoadError: AssertionError: f(a) * f(b) < 0

In [84]:
marketClearing2(0.000000001)

9.977304162371334e-9

Only a price of zero will give market clearing?!